In [2]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12, is_inf
import numpy as np
import galois 
from functools import reduce
GF = galois.GF(curve_order)

In [64]:
# First we evaluate Groth16 with unencrypted values to verify proof & verify operations
# We then create encrypted values and evaluate Groth16 with encrypted values to verify proof & verify operations
# 
# 
# 
# R1CS
# 
# circuit definition for proving knowledge of  x^3 + 2x^3y - 5xy^2 -3y + 2
# in R1CS world, we have L * R = O with m x n matrices and m length witness vector
L = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,-5,0,0,0,0,0]
    ])

R = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ])

O = np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [-2,1,0,3,0,-1,0,-2]
    ])

r1cs_m, r1cs_n  = L.shape[1], L.shape[0] #m columns, n rows
x = GF(5)
y = GF(10)
v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print("out: ",int(out))

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"

# EC mul scaler, checks for Identity elem
def ecmul(point, scalar):
    if(is_inf(point) or scalar == 0):
        return Z1
    return multiply(point, mod_scalar(int(scalar)))

# EC Add function
# accepts either
#  - 2 points => adds 2 points
#  - 1 point and 1 scalar => multiplies point by scalar
#  - 1 point and 1 array of scalars => multiplies point by each scalar and adds them together
def ecadd(s, *args):
    if(s == Z1) and not args:
        return Z1
    
    for x in args:
        if x == Z1:
            continue #skip adding Z1
        if isinstance(x, tuple) and len(x) == 2:
            # Directly add if x is a point on the curve
            s = add(s, x)
        else:
            # Assume x is a scalar and try multiplying with G1
            try:
                s = add(s, ecmul(G1, x))
            except Exception:
                # If it fails, try with G2
                s = add(s, ecmul(G2, x))
    return s

# mods to curve_order incase a > curve_order
def mod_scalar(a):
    return int(a % curve_order)

# mods array to curve_order
def mod_array(a):
    return np.array([mod_scalar(u) for u in a.flatten()]).reshape(a.shape)

# convert numpy arrays to galois arrays
L_galois, R_galois, O_galois = GF(mod_array(L)), GF(mod_array(R)), GF(mod_array(O))


# Quadratic Arithmetic Program (QAP)
# 
# QAP translates R1CS into polynomials. Succinctness is derived from interpolating polynomials using Lagrange
# representing constraints as a single polynomial. This is possible because vector addition is homomorphic to polynomial addition

# generate target polynomial with roots 1,2,3,4,5 i.e. (x-1)(x-2)(x-3)(x-4)(x-5)
tx_roots = GF([1,2,3,4,5])
tx_poly = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots(tx_roots)).astype(int),  field=GF)

# Polynomial Functions
# interpolate columns of matrix with lagrange interpolation and target polynomial
def interpolate_columns(col):
    return galois.lagrange_poly(tx_roots, col)

# convert matrix to polynomials by interpolated the columns. Matrix is m columns, hence, m polynomials
def matrix_to_polys(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

# multiply each polynomial by the witness value at that index i.e. poly[0] * witness[0]
def mul_polys_by_witness(polys, witness):
    return [poly * int(witness[i]) for i, poly in enumerate(polys)]

# vector addition is homomorphic to polynomial addition. Hence, we can reduce the vector of polynomials to one polynomial
def reduce_polys_to_one(polys, start_index, end_index):
    return reduce(lambda a, b: a + b, polys[start_index:end_index])

# split polynomials into public and private parts
def split_polys(polys, split_index):
    return reduce_polys_to_one(polys, 0, split_index), reduce_polys_to_one(polys, split_index, len(polys))

# convert L matrix to U polynomials. L is m x n matrix, U will have m polynomials
U_polys, V_polys, W_polys = matrix_to_polys(L_galois), matrix_to_polys(R_galois), matrix_to_polys(O_galois)
Ua_polys, Va_polys, Wa_polys = mul_polys_by_witness(U_polys, witness), mul_polys_by_witness(V_polys, witness), mul_polys_by_witness(W_polys, witness)
Ua_poly, Va_poly, Wa_poly = reduce_polys_to_one(Ua_polys, 0, r1cs_m), reduce_polys_to_one(Va_polys, 0, r1cs_m), reduce_polys_to_one(Wa_polys, 0, r1cs_m)

hx_poly, remainder = divmod(((Ua_poly * Va_poly) - Wa_poly), tx_poly)
assert remainder == 0, "remainder is not zero"
hx_tx_poly = hx_poly * tx_poly

# Trusted Setup parameters for Groth16
# 
# tau used for powers of tau ceremony
# alpha, beta used for shifting polynomials and prevent prover from faking proof values
# gamma, delta 
# 
# notice we are not using powers of tau or encrypted values of alpha, beta, gamma, delta till
# 
# 
def setup_trusted_parameters():
    tau, alpha, beta, gamma, delta = [GF(i) for i in (20, 2, 3, 4, 5)]
    gamma_inv, delta_inv = [pow(int(x), -1, curve_order) for x in (gamma, delta)]
    gamma_invg1, delta_invg1 = ecmul(G1, int(gamma_inv)), ecmul(G1, int(delta_inv))
    return tau, alpha, beta, gamma, delta, gamma_inv, delta_inv, gamma_invg1, delta_invg1

tau, alpha, beta, gamma, delta, gamma_inv, delta_inv, gamma_invg1, delta_invg1 = setup_trusted_parameters()

alpha_deltainv = ecmul(G1, int(alpha * delta_inv))
beta_deltainv = ecmul(G1, int(beta * delta_inv))
alpha_gammainv = ecmul(G1, int(alpha * gamma_inv))
beta_gammainv = ecmul(G1, int(beta * gamma_inv))

alphag1 = ecmul(G1, alpha)
betag1 = ecmul(G1, beta)
betag2 = ecmul(G2, beta)
deltag1 = ecmul(G1, delta)
deltag2 = ecmul(G2, delta)
gammag1 = ecmul(G1, gamma)
gammag2 = ecmul(G2, gamma)

public_witness_length = 2

r = GF(3)
s = GF(4)

# evaluate polynomials at tau
ua_tau, va_tau, wa_tau = int(Ua_poly(tau)), int(Va_poly(tau)), int(Wa_poly(tau))
hx_tx_poly_tau = int(hx_tx_poly(tau))

# split polynomials into public and private parts
ua_pub_poly, ua_priv_poly = split_polys(Ua_polys, public_witness_length)
va_pub_poly, va_priv_poly = split_polys(Va_polys, public_witness_length)
wa_pub_poly, wa_priv_poly = split_polys(Wa_polys, public_witness_length)
    
ua_private, va_private, wa_private = int(ua_priv_poly(tau)), int(va_priv_poly(tau)), int(wa_priv_poly(tau))
ua_public, va_public, wa_public = int(ua_pub_poly(tau)), int(va_pub_poly(tau)), int(wa_pub_poly(tau)) 

# Generate Proofs, Verify Pairings
A1 = ecadd(alphag1, ua_tau, ecmul(deltag1,r))
B2 = ecadd(betag2, va_tau, ecmul(deltag2, s))
B1 = ecadd(betag1, va_tau, ecmul(deltag1, s))

C1_private = ecadd(
    ecmul(beta_deltainv, ua_private), 
    ecmul(alpha_deltainv, va_private),
    ecmul(delta_invg1, wa_private),
    ecmul(delta_invg1, hx_tx_poly_tau), 
    ecmul(A1, int(s)), 
    ecmul(B1, int(r)), 
    ecmul(deltag1, int(-r * s))
)

C1_public = ecadd(
    ecmul(beta_gammainv, ua_public),
    ecmul(alpha_gammainv, va_public),
    ecmul(gamma_invg1, wa_public),
)

AB = pairing(B2,neg(A1))
ab = pairing(betag2,alphag1)
CD = pairing(gammag2, C1_public)
EF = pairing(deltag2, C1_private)

# Coeffs AB = Coeffs ab * CD * EF
final_exponentiate(AB * ab * CD * EF) == FQ12.one()

out:  97


True

In [65]:
# Trusted Setup for encrypted G16
def powers_of_tau(tau, n, G):
    return [ecmul(G, int(tau) ** i) for i in range(n)]

def generate_ptau_for_c(U_polys, V_polys, W_polys, beta, alpha, tau):
    return [ecmul(G1, int(beta * U_polys[i](tau) + alpha * V_polys[i](tau) + W_polys[i](tau))) for i in range(0, len(Ua_polys))]

def inner_product(powers_of_tau, poly_coeffs):
    return reduce(ecadd, (ecmul(point, int(coeff)) for point, coeff in zip(powers_of_tau, poly_coeffs)), Z1)

# Trusted Setup values for encrypted G16
# generate powers of tau for G1, G2, target polynomial, and product of sum(Beta * U + Alpha * V + W)
p_taug1 = powers_of_tau(tau, r1cs_m, G1)
p_taug2 = powers_of_tau(tau, r1cs_m, G2)
tx_p_taug1 = powers_of_tau(tau, tx_poly.degree, G1)
tx_tau = tx_poly(int(tau))
tx_p_taug1_tau = [ecmul(tx_p_taug1[i], int(tx_tau)) for i in range(0, len(tx_p_taug1))]
bui_avi_wi_tau = generate_ptau_for_c(U_polys, V_polys, W_polys, beta, alpha, tau)

# evaluate polynomials at powers of tau, hiding private values
Ua_taug1 = inner_product(p_taug1, Ua_poly.coeffs[::-1])
Va_taug2 = inner_product(p_taug2, Va_poly.coeffs[::-1])
Va_taug1 = inner_product(p_taug1, Va_poly.coeffs[::-1])
Wa_taug1 = inner_product(p_taug1, Wa_poly.coeffs[::-1])
hx_tx_taug1 = inner_product(tx_p_taug1_tau, hx_poly.coeffs[::-1])
ca_taug1 = inner_product(bui_avi_wi_tau, witness)

A = ecadd(alphag1, Ua_taug1)
B = ecadd(betag2, Va_taug2)
C = ecadd(ca_taug1, hx_tx_taug1)

AB = pairing(B,neg(A))
ab = pairing(betag2,alphag1)
CD = pairing(G2, C)

# Verify encrypted G16 proof, preliminary check
final_exponentiate(AB * ab * CD) == FQ12.one()

True

In [66]:
# get 2 points from bui_avi_wi_tau (it doesn't contain witness or gamma/delta or ht)
# 
# Here we split bui_avi_wi_tau into public and private parts. We do this because the witness is actually split between 
# public and private parts. The public part is generated at trusted setup, the private part is generated by prover
# 
bui_avi_wi_tau_gammainv_pub = [ecmul(point, int(gamma_inv)) for point in bui_avi_wi_tau[:public_witness_length]]
bui_avi_wi_tau_gammainv_witness_pub = inner_product(bui_avi_wi_tau_gammainv_pub, witness[:public_witness_length])

bui_avi_wi_tau_deltainv_priv = [ecmul(point, int(delta_inv)) for point in bui_avi_wi_tau[public_witness_length:]]
bui_avi_wi_tau_deltainv_witness_priv = inner_product(bui_avi_wi_tau_deltainv_priv, witness[public_witness_length:])

# 
hx_tx_taug1_deltainv = ecmul(hx_tx_taug1, int(delta_inv))

AB = pairing(B,neg(A))
ab = pairing(betag2,alphag1)
C_pubG = pairing(gammag2, bui_avi_wi_tau_gammainv_witness_pub)
C_privD_ht = ecadd(hx_tx_taug1_deltainv, bui_avi_wi_tau_deltainv_witness_priv)
C_privD = pairing(deltag2, C_privD_ht)

final_exponentiate(AB * ab * C_pubG * C_privD) == FQ12.one()

True

In [67]:
# prover adds r, s to hide private values
A = ecadd(alphag1, Ua_taug1, ecmul(deltag1,r))
B1 = ecadd(betag1, Va_taug1, ecmul(deltag1, s))
B2 = ecadd(betag2, Va_taug2, ecmul(deltag2, s))

C_pub = bui_avi_wi_tau_gammainv_witness_pub # no change

# add r,s to hide private values
C_priv = ecadd( 
    bui_avi_wi_tau_deltainv_witness_priv,
    hx_tx_taug1_deltainv,
    ecmul(A, int(s)),
    ecmul(B1, int(r)),
    ecmul(deltag1, int(-r * s))
)


AB = pairing(B2,neg(A))
ab = pairing(betag2,alphag1)
C_pubG = pairing(gammag2, C_pub)
C_privD = pairing(deltag2, C_priv)

final_exponentiate(AB * ab * C_pubG * C_privD) == FQ12.one()

True